In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from tqdm import tqdm
import random
from itertools import product

# 1. 데이터 생성 및 전처리 과정

## 1-1 기존 데이터 (user_course에 학과, 과목 삭제 조건 적용)

In [2]:
# data initialization
user_major = {}
category_course = {}
major_category = pd.read_csv('./data/major_category.csv', encoding='CP949', header=None)
major_course={}
userlist=[]
courselist=[]
elelist=[]
ele_course=[]

# hakno re
coursere=re.compile('^[A-Z]{3}\d*')

# category, deluniv input
with open('./data/catlist.txt', encoding='UTF-8') as f:
    valid_category = [cate.split("\t")[0] for cate in f.readlines()]
with open('./data/elelist.txt', encoding='UTF-8') as f:
    elelist = [cate.split("\t")[0] for cate in f.readlines()]
with open('./data/deluniv.txt', encoding='UTF-8') as f:
    deluniv = [univ[:-1] for univ in f.readlines()]
    
# data creation
user_course = pd.read_csv('./data/course_userdata.csv', low_memory=False, header=None)
for row in user_course.values:
    if (row[1] in deluniv or str(row[4]) not in valid_category or re.match(coursere, row[5]) == None):
        continue
    user = row[0]
    major = row[1] + '_' + row[2]
    category = row[4]
    course = re.search(coursere, row[5]).group()
    year = row[6]
    term = row[7]
    if user not in user_major:
        user_major[user] = [major]
        userlist.append(user)
    elif major not in user_major[user]:
        user_major[user].append(major)

    if category not in category_course:
        category_course[category] = [course]
    elif course not in category_course[category]:
        category_course[category].append(course)

    if (category in elelist and course not in ele_course):
        ele_course.append(course)

    if course not in courselist:
        courselist.append(course)

for row in major_category.values:
    if row[0] not in major_course:
        major_course[row[0]] = category_course[row[1]]
    else:
        major_course[row[0]] += category_course[row[1]]

In [3]:
print(len(user_course))
user_course.head()

1920266


,0,1,2,3,4,5,6,7,8
0,11900,문과대학,사학,현대사회와심리학,26,UF122-09,2003,1,문과대학
1,71964,음악대학,기악과,현대사회와심리학,26,UF122-09,2003,1,문과대학
2,50516,교육과학대학,교육학과,현대사회와심리학,26,UF122-09,2003,1,문과대학
3,232,이과대학,천문우주학,현대사회와심리학,26,UF122-09,2003,1,문과대학
4,71826,공과대학,세라믹공학,현대사회와심리학,26,UF122-09,2003,1,문과대학


## 1-2 학습을 위한 추가 전처리

In [4]:
# 데이터 수정을 위해 컬럼명 지정
user_course.columns = ['a','b','c','d','e','f','g','h','i']

#a:삭제 대상 학과, b:고려대상 과목, c:조건에 해당하는 user, d:15,16,17년도 각각 조건 적용
a = ~user_course.b.isin(deluniv)
b = user_course.f.str.contains(coursere)
c = user_course.a.isin(userlist)
d = user_course.g.isin([2015,2016,2017])
user_course = user_course[a & b & c & d]

#j 컬럼에 학정번호 앞부분 저장(분반제거) ex) CEE4402-01 > CEE4402
user_course['j'] = user_course['f'].str.findall(coursere)
user_course['j'] = user_course.f.str.split('-').str[0]
user_course = user_course[user_course['j'].isin(courselist)]

#17-2에 수업 들었던 학생(user_20172), 15,16,17-1에 수업 들었던 학생(user_left)
#17년 2월에 수업 들었던 학생 중 나머지 학기에도 들었던 학생(user_fit)리스트 저장
user_20172 = list(set(user_course[(user_course['g'] == 2017) & (user_course['h'] == 2)]['a'].tolist()))
user_left = list(set(user_course[~((user_course['g'] == 2017) & (user_course['h'] == 2))]['a'].tolist()))
user_fit = []
for i in user_20172:
    if i in user_left:
        user_fit.append(i)

#user_fit에 해당하는 학생들의 데이터만 남겨서 user_course에 filtering
user_course = user_course[user_course.a.isin(user_fit)]
        
#2017년 2학기 개강한 수업(course_20172_list)
course_20172_list = list(set(user_course[(user_course['g'] == 2017) & (user_course['h'] == 2)]['j'].tolist()))

print("17-2에 수업 들었던 학생 : {}".format(len(user_20172)))
print("15,16,17-1에 수업 들었던 학생 : {}".format(len(user_left)))
print("17년 2월에 수업 들었던 학생 중 나머지 학기에도 들었던 학생 : {}".format(len(user_fit)))

17-2에 수업 들었던 학생 : 12271
15,16,17-1에 수업 들었던 학생 : 26906
17년 2월에 수업 들었던 학생 중 나머지 학기에도 들었던 학생 : 12031


In [5]:
# user_course (1920266 > 285482)
print(len(user_course))
user_course.head()

285482


,a,b,c,d,e,f,g,h,i,j
564948,154440,공과대학,토목·환경공학,구조시스템설계,179,CEE4402-01,2015,1,토목환경공학전공,CEE4402
564964,139425,공과대학,토목·환경공학,구조시스템설계,179,CEE4402-01,2015,1,토목환경공학전공,CEE4402
564969,190801,공과대학,화공생명공학,공학수학(3),179,MAT2016-07,2015,1,토목환경공학전공,MAT2016
564973,189376,공과대학,기계공학,공학수학(3),179,MAT2016-07,2015,1,토목환경공학전공,MAT2016
564975,188577,공과대학,토목·환경공학,공학수학(3),179,MAT2016-07,2015,1,토목환경공학전공,MAT2016


# 2. NCF 학습

## 2-1 학습 데이터 생성

user_course['user_id'] = user_course['a'].astype("category").cat.codes
user_course['item_id'] = user_course['j'].astype("category").cat.codes
df_to_node2vec = user_course[['a','b','c','d','j','i','g','h','user_id','item_id']].copy()
df_to_node2vec.columns = ['user', 'major1', 'major2', 'item_name', 'item_num', 'item_major', 'year', 'semester', 'user_id', 'item_id']
df_to_node2vec = df_to_node2vec.reset_index(drop=True)

##### 데이터 전달
df_to_node2vec.to_csv('./data/df_to_node2vec.csv',encoding="CP949")

In [6]:
user_course_inter_df = user_course[['a','j','g','h']].copy()
user_course_inter_df.columns = ['user','item','year','sem']
user_course_inter_df = user_course_inter_df.reset_index(drop=True)

# item_id 부여 + user_id column 추가
user_course_inter_df['user_id'] = user_course_inter_df['user'].astype("category").cat.codes
user_course_inter_df['item_id'] = user_course_inter_df['item'].astype("category").cat.codes

users = list(np.sort(user_course_inter_df.user_id.unique()))
items = list(np.sort(user_course_inter_df.item_id.unique()))

user_course_train = user_course_inter_df[~((user_course_inter_df['year'] == 2017) & (user_course_inter_df['sem'] == 2))]
user_course_test = user_course_inter_df[(user_course_inter_df['year'] == 2017) & (user_course_inter_df['sem'] == 2)]

print("# of train users : {}".format(len(users)))
print("# of train items : {}".format(len(items)))
print("# of interactions : {}".format(len(user_course_inter_df)))


print("train inter : {}".format(len(user_course_train)))
print("test inter : {}".format(len(user_course_test)))

# of train users : 12031
# of train items : 2987
# of interactions : 285482
train inter : 220708
test inter : 64774


## 2-2 negative case 생성

In [7]:
import random

In [8]:
neg_num=4
neg_full_u = []
neg_full_i = []

for u in tqdm(users):
    train_num = len(user_course_train[user_course_train['user_id']==u])
    full = {i for i in range(len(items))}
    asis = set(user_course_inter_df[user_course_inter_df['user_id']==u]['item_id'].tolist())
    
    temp_neg_u = [u for i in range(train_num*neg_num)]
    temp_neg_i = random.sample(list(full-asis), train_num*neg_num)
    
    neg_full_u.extend(temp_neg_u)
    neg_full_i.extend(temp_neg_i)  

100%|███████████████████████████████████████████████████████████████████████████| 12031/12031 [00:16<00:00, 733.14it/s]


## 2-3 train_df 만들기

In [9]:
u_df = user_course_train['user_id'].tolist()
u_df.extend(neg_full_u)

i_df = user_course_train['item_id'].tolist()
i_df.extend(neg_full_i)

p_df = [1 for i in range(len(user_course_train))]
p_df.extend([0 for i in range(len(neg_full_u))])

print(len(u_df),len(i_df),len(p_df))

train_df = pd.DataFrame({'user_id':u_df, 'item_id':i_df, 'plays':p_df})

1103540 1103540 1103540


## 모델 인풋 생성 + shuff

In [10]:
# train user, item 리스트 생성
rows = train_df['user_id'].astype(int)
cols = train_df['item_id'].astype(int)
values = list(train_df.plays)

uids = np.array(rows.tolist())
iids = np.array(cols.tolist())

user_input = uids.tolist()
item_input = iids.tolist()
labels = values

user_data_shuff, item_data_shuff, label_data_shuff = shuffle(user_input, item_input, labels)
user_data_shuff = np.array(user_data_shuff).reshape(-1,1)
item_data_shuff = np.array(item_data_shuff).reshape(-1,1)
label_data_shuff = np.array(label_data_shuff).reshape(-1,1)

print(len(user_data_shuff))
print(len(item_data_shuff))
print(len(label_data_shuff))

1103540
1103540
1103540


In [11]:
user_course_train

,user,item,year,sem,user_id,item_id
0,154440,CEE4402,2015,1,258,430
1,139425,CEE4402,2015,1,173,430
2,190801,MAT2016,2015,1,2206,1795
3,189376,MAT2016,2015,1,1922,1795
4,188577,MAT2016,2015,1,1617,1795
...,...,...,...,...,...,...
283360,323663,UCB1104,2017,1,9852,2880
283361,350112,UCB1104,2017,1,11962,2880
283362,348964,UCB1104,2017,1,11808,2880
283363,348965,UCB1104,2017,1,11809,2880


# 모델

In [12]:
from model.NeuMF import NeuMF

In [13]:
nmf64 = NeuMF(len(users), len(items), emb_size=64)
model=nmf64.get_model()

Tensor("flatten/Reshape:0", shape=(None, 64), dtype=float32)
Tensor("flatten_1/Reshape:0", shape=(None, 64), dtype=float32)
Tensor("flatten_2/Reshape:0", shape=(None, 64), dtype=float32)
Tensor("flatten_3/Reshape:0", shape=(None, 64), dtype=float32)


In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 64)        769984      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 64)        191168      input_2[0][0]                    
_______________________________________________________________________________________

In [15]:
model.fit([user_data_shuff, item_data_shuff], label_data_shuff, epochs=20,
               batch_size=256, verbose=1)

Epoch 1/20
4311/4311 [==============================] - 58s 13ms/step - loss: 0.3182
Epoch 2/20
4311/4311 [==============================] - 50s 12ms/step - loss: 0.1268
Epoch 3/20
4311/4311 [==============================] - 52s 12ms/step - loss: 0.0816
Epoch 4/20
4311/4311 [==============================] - 50s 12ms/step - loss: 0.0600
Epoch 5/20
4311/4311 [==============================] - 51s 12ms/step - loss: 0.0461
Epoch 6/20
4311/4311 [==============================] - 50s 12ms/step - loss: 0.0390
Epoch 7/20
4311/4311 [==============================] - 50s 12ms/step - loss: 0.0365
Epoch 8/20
4311/4311 [==============================] - ETA: 0s - loss: 0.035 - 50s 12ms/step - loss: 0.0357
Epoch 9/20
4311/4311 [==============================] - 50s 12ms/step - loss: 0.0353 0s - 
Epoch 10/20
4311/4311 [==============================] - 50s 12ms/step - loss: 0.0352
Epoch 11/20
4311/4311 [==============================] - 52s 12ms/step - loss: 0.0351
Epoch 12/20
4311/4311 [==========

# Predict

In [16]:
user_dict = dict(zip(user_course_inter_df['user'].tolist(), user_course_inter_df['user_id'].tolist()))
item_dict = dict(zip(user_course_inter_df['item'].tolist(), user_course_inter_df['item_id'].tolist()))

In [17]:
model.save('./pretrain/base-ncf-64.h5')

In [18]:
def predict_return(u, K):
    user=user_course_inter_df[user_course_inter_df['user_id']==u]['user'].iloc[0]

    full = set(user_course_test['item_id'].tolist())
    asis = set(user_course_train[user_course_train['user_id']==u]['item_id'].tolist())
    pred_cand_list = list(full-asis)
    #print(len(full),len(asis),len(pred_cand_list))
    pred_user_list = [u for i in range(len(pred_cand_list))]

    pred_user = np.array(pred_user_list).reshape(-1,1)
    pred_cand = np.array(pred_cand_list).reshape(-1,1)

    # predict 진행
    predictions = model.predict([pred_user, pred_cand])
    predictions = predictions.flatten().tolist()

    pred_dict = dict(zip(pred_cand_list, predictions))
    pred_df = pd.DataFrame({'item':pred_cand_list, 'score':predictions})

    # 유저에 해당되는 전공 과목 리스트 가져옴
    temp_user_major = major_course[user_major[user][0]]
    for i in range(len(temp_user_major)):
        try:
            temp_user_major[i] = item_dict[temp_user_major[i]]
        except:
            pass
    #temp_user_major = list(map(lambda x:item_dict[x],temp_user_major))

    # 전공, 비전공 분류
    req_df = pred_df[pred_df['item'].isin(temp_user_major)]
    fre_df = pred_df[~pred_df['item'].isin(temp_user_major)]

    # 예측결과 정렬해서 list 형태 출력
    req_list = req_df.sort_values(by='score',ascending=False)['item'].tolist()
    fre_list = fre_df.sort_values(by='score',ascending=False)['item'].tolist()

    return(req_list[0:K], fre_list[0:K])

In [19]:
# user_id, K input으로
predict_return(1,5)

([1457, 1456, 1458, 1500, 1484], [2968, 1185, 2537, 1027, 252])

# Evaluate

In [20]:
import numpy as np

def recall_(p,r):
    try:
        return(int(sum(p))/int(len(r)))
    except:
        return(0)

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    #print(r)
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)


def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k=20, method=1):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [21]:
def asis_return(u):
    user=user_course_inter_df[user_course_inter_df['user_id']==u]['user'].iloc[0]

    # 유저에 해당되는 전공 과목 리스트 가져옴
    temp_user_major = major_course[user_major[user][0]]
    for i in range(len(temp_user_major)):
        try:
            temp_user_major[i] = item_dict[temp_user_major[i]]
        except:
            pass
    #temp_user_major = list(map(lambda x:item_dict[x],temp_user_major))

    temp_asis_df = user_course_test[user_course_test['user_id']==u].copy()

    req_asis = temp_asis_df[temp_asis_df['item_id'].isin(temp_user_major)]
    fre_asis = temp_asis_df[~temp_asis_df['item_id'].isin(temp_user_major)]

    return(req_asis['item_id'].tolist(), fre_asis['item_id'].tolist())

In [22]:
# user_id 만 집어넣음
asis_return(1)

([1540, 1514, 1506, 1484, 1459, 1492], [2980, 1026])

In [23]:
p=[]
r=[]

for i in tqdm(range(12031)):
    eval_gt = asis_return(i)
    eval_pred = predict_return(i,10) # pred개수

    temp_req=[1 if c in eval_gt[0] else 0 for c in eval_pred[0]]
    #temp_pre=[1 if c in eval_gt[1] else 0 for c in eval_pred[1]]

    #precision_at_k(temp_req,10)
    
    # 앞에 temp, 뒤에 gt
    #recall_(temp_req, eval_gt[0])
    if eval_gt[0] != []:
        p.append(precision_at_k(temp_req,len(temp_req)))
        r.append(recall_(temp_req, eval_gt[0]))

100%|████████████████████████████████████████████████████████████████████████████| 12031/12031 [13:48<00:00, 14.52it/s]


In [24]:
print(sum(p)/len(p))
print(sum(r)/len(r))

0.25484389256951806
0.6319921832031801
